# DuckDB with S3/Wasabi Storage

This notebook demonstrates how to:
- Configure DuckDB to work with S3/Wasabi
- Save data to S3 in Parquet format
- Read JSON from APIs and save to S3
- Query data directly from S3
- Create data marts

## Setup: Configure DuckDB with S3 Credentials

In [ ]:
import duckdb
import os
import json

# Get Wasabi credentials from environment
wasabi_endpoint = os.getenv('WASABI_ENDPOINT', 's3.us-east-2.wasabisys.com')
wasabi_access_key = os.getenv('WASABI_ACCESS_KEY')
wasabi_secret_key = os.getenv('WASABI_SECRET_KEY')

# Create DuckDB connection
con = duckdb.connect()

# Configure S3 settings
con.execute(f"""
    SET s3_endpoint='{wasabi_endpoint}';
    SET s3_access_key_id='{wasabi_access_key}';
    SET s3_secret_access_key='{wasabi_secret_key}';
    SET s3_url_style='path';
""")

print("✓ DuckDB configured with S3 credentials")
print(f"Endpoint: {wasabi_endpoint}")

## Example 1: Write DataFrame to S3 as Parquet

In [ ]:
# Create sample data
sample_data = [
    {'id': 1, 'name': 'Alice', 'age': 25, 'department': 'Engineering'},
    {'id': 2, 'name': 'Bob', 'age': 30, 'department': 'Marketing'},
    {'id': 3, 'name': 'Charlie', 'age': 35, 'department': 'Sales'},
    {'id': 4, 'name': 'Diana', 'age': 28, 'department': 'Engineering'},
    {'id': 5, 'name': 'Eve', 'age': 32, 'department': 'HR'}
]

# Create table from Python data
con.execute("""
    CREATE OR REPLACE TABLE employees AS 
    SELECT * FROM (VALUES
        (1, 'Alice', 25, 'Engineering'),
        (2, 'Bob', 30, 'Marketing'),
        (3, 'Charlie', 35, 'Sales'),
        (4, 'Diana', 28, 'Engineering'),
        (5, 'Eve', 32, 'HR')
    ) AS t(id, name, age, department)
""")

# View the data
con.execute("SELECT * FROM employees").df()

In [ ]:
# Write to S3 as Parquet
bucket_name = "dfscrunch-data-lake"
output_path = f"s3://{bucket_name}/duckdb-examples/employees.parquet"

con.execute(f"""
    COPY employees 
    TO '{output_path}' 
    (FORMAT PARQUET, COMPRESSION 'SNAPPY')
""")

print(f"✓ Data written to: {output_path}")

## Example 2: Read JSON from API and Save to S3

This example shows how to:
1. Fetch JSON data from an API
2. Load it into DuckDB
3. Transform it
4. Save to S3 as Parquet

In [ ]:
# Simulate API response (replace with actual API call)
# In real scenario: response = requests.get('https://api.example.com/data').json()

api_response = {
    "users": [
        {"user_id": 101, "username": "john_doe", "email": "john@example.com", "created_at": "2024-01-15"},
        {"user_id": 102, "username": "jane_smith", "email": "jane@example.com", "created_at": "2024-02-20"},
        {"user_id": 103, "username": "bob_jones", "email": "bob@example.com", "created_at": "2024-03-10"},
    ],
    "metadata": {
        "total": 3,
        "timestamp": "2024-10-05T20:00:00Z"
    }
}

print("API Response:")
print(json.dumps(api_response, indent=2))

In [ ]:
# Method 1: Load JSON directly into DuckDB
# Save JSON to temporary file (or use read_json_auto for files)
import tempfile

with tempfile.NamedTemporaryFile(mode='w', suffix='.json', delete=False) as f:
    json.dump(api_response['users'], f)
    temp_json_path = f.name

# Read JSON file into DuckDB
users_df = con.execute(f"""
    SELECT 
        user_id,
        username,
        email,
        CAST(created_at AS DATE) as created_at,
        CURRENT_TIMESTAMP as ingested_at
    FROM read_json_auto('{temp_json_path}')
""").df()

print("Transformed data:")
print(users_df)

# Clean up temp file
os.unlink(temp_json_path)

In [ ]:
# Method 2: Load from Python object via pandas
# Convert to pandas DataFrame first, then DuckDB can read it

import pandas as pd

# Convert API response to DataFrame
users_df = pd.DataFrame(api_response['users'])

# Create table from DataFrame
con.execute("""
    CREATE OR REPLACE TABLE api_users AS 
    SELECT 
        user_id,
        username,
        email,
        CAST(created_at AS DATE) as created_at,
        CURRENT_TIMESTAMP as ingested_at
    FROM users_df
""")

con.execute("SELECT * FROM api_users").df()

In [ ]:
# Save API data to S3 as Parquet
api_output_path = f"s3://{bucket_name}/duckdb-examples/api_users.parquet"

con.execute(f"""
    COPY api_users 
    TO '{api_output_path}' 
    (FORMAT PARQUET, COMPRESSION 'SNAPPY')
""")

print(f"✓ API data saved to: {api_output_path}")

## Example 3: Query Parquet Files Directly from S3

DuckDB can query Parquet files on S3 without loading them into memory!

In [ ]:
# Query S3 data directly
result = con.execute(f"""
    SELECT 
        department,
        COUNT(*) as employee_count,
        AVG(age) as avg_age,
        MIN(age) as min_age,
        MAX(age) as max_age
    FROM read_parquet('s3://{bucket_name}/duckdb-examples/employees.parquet')
    GROUP BY department
    ORDER BY employee_count DESC
""").df()

print("Department Statistics:")
print(result)

In [ ]:
# Query multiple S3 files with wildcard
# (useful for partitioned data)
result = con.execute(f"""
    SELECT COUNT(*) as total_records
    FROM read_parquet('s3://{bucket_name}/duckdb-examples/*.parquet')
""").df()

print(f"Total records across all files: {result['total_records'][0]}")

## Example 4: Create a Data Mart

Combine multiple datasets to create an analytical mart

In [ ]:
# Create a mart by joining data from S3
mart = con.execute(f"""
    WITH employee_summary AS (
        SELECT 
            department,
            COUNT(*) as total_employees,
            AVG(age) as avg_age
        FROM read_parquet('s3://{bucket_name}/duckdb-examples/employees.parquet')
        GROUP BY department
    ),
    user_summary AS (
        SELECT 
            COUNT(*) as total_users,
            COUNT(DISTINCT DATE_TRUNC('month', created_at)) as active_months
        FROM read_parquet('s3://{bucket_name}/duckdb-examples/api_users.parquet')
    )
    SELECT 
        e.department,
        e.total_employees,
        ROUND(e.avg_age, 1) as avg_age,
        u.total_users,
        u.active_months,
        CURRENT_TIMESTAMP as mart_created_at
    FROM employee_summary e
    CROSS JOIN user_summary u
    ORDER BY e.total_employees DESC
""").df()

print("Data Mart:")
print(mart)

In [ ]:
# Save mart back to S3
mart_output_path = f"s3://{bucket_name}/ddm/employee_user_summary.parquet"

con.execute(f"""
    COPY (
        WITH employee_summary AS (
            SELECT 
                department,
                COUNT(*) as total_employees,
                AVG(age) as avg_age
            FROM read_parquet('s3://{bucket_name}/duckdb-examples/employees.parquet')
            GROUP BY department
        ),
        user_summary AS (
            SELECT 
                COUNT(*) as total_users,
                COUNT(DISTINCT DATE_TRUNC('month', created_at)) as active_months
            FROM read_parquet('s3://{bucket_name}/duckdb-examples/api_users.parquet')
        )
        SELECT 
            e.department,
            e.total_employees,
            ROUND(e.avg_age, 1) as avg_age,
            u.total_users,
            u.active_months,
            CURRENT_TIMESTAMP as mart_created_at
        FROM employee_summary e
        CROSS JOIN user_summary u
    )
    TO '{mart_output_path}'
    (FORMAT PARQUET, COMPRESSION 'SNAPPY')
""")

print(f"✓ Mart saved to: {mart_output_path}")

## Example 5: Working with Real API Data

Here's a practical example with a real API (JSONPlaceholder)

In [ ]:
# Install requests if needed
# !pip install requests

import requests

# Fetch data from a real API
response = requests.get('https://jsonplaceholder.typicode.com/users')
users_api_data = response.json()

print(f"Fetched {len(users_api_data)} users from API")
print("Sample record:")
print(json.dumps(users_api_data[0], indent=2))

In [ ]:
# Flatten nested JSON and load into DuckDB
flattened_users = [
    {
        'user_id': u['id'],
        'name': u['name'],
        'username': u['username'],
        'email': u['email'],
        'city': u['address']['city'],
        'company': u['company']['name'],
    }
    for u in users_api_data
]

# Convert to DataFrame
flattened_df = pd.DataFrame(flattened_users)

# Create table from DataFrame - DuckDB reads pandas DataFrames directly!
con.execute("""
    CREATE OR REPLACE TABLE real_api_users AS 
    SELECT 
        *,
        CURRENT_TIMESTAMP as ingested_at
    FROM flattened_df
""")

# View the data
con.execute("SELECT * FROM real_api_users LIMIT 5").df()

In [ ]:
# Save to S3
real_api_output = f"s3://{bucket_name}/duckdb-examples/real_api_users.parquet"

con.execute(f"""
    COPY real_api_users 
    TO '{real_api_output}' 
    (FORMAT PARQUET, COMPRESSION 'SNAPPY')
""")

print(f"✓ Real API data saved to: {real_api_output}")

## Cleanup

## Example 7: Schema Inspection (Like Spark's printSchema)

DuckDB has multiple ways to inspect schemas - some even better than Spark!

In [ ]:
# Method 1: DESCRIBE - Most like Spark's printSchema()
print("=" * 60)
print("Method 1: DESCRIBE (like Spark's df.printSchema())")
print("=" * 60)

schema = con.execute("DESCRIBE employees").df()
print(schema)
print("\nColumns:")
for _, row in schema.iterrows():
    print(f"  {row['column_name']}: {row['column_type']}")

In [ ]:
# Method 2: SUMMARIZE - Schema + Statistics (DuckDB exclusive!)
# This is BETTER than Spark - you get schema AND data stats!
print("\n" + "=" * 60)
print("Method 2: SUMMARIZE (schema + stats - unique to DuckDB!)")
print("=" * 60)

summary = con.execute("SUMMARIZE employees").df()
print(summary.to_string())

In [ ]:
# Method 3: DESCRIBE for S3 Parquet files (without loading!)
# This is AMAZING - inspect schema of S3 files without downloading!
print("\n" + "=" * 60)
print("Method 3: DESCRIBE S3 Parquet (no download needed!)")
print("=" * 60)

s3_schema = con.execute(f"""
    DESCRIBE SELECT * FROM read_parquet('s3://{bucket_name}/duckdb-examples/employees.parquet')
""").df()

print(s3_schema)

In [ ]:
# Method 4: PRAGMA table_info - Detailed metadata
print("\n" + "=" * 60)
print("Method 4: PRAGMA table_info (detailed metadata)")
print("=" * 60)

table_info = con.execute("PRAGMA table_info('employees')").df()
print(table_info)

In [ ]:
# Method 5: Python .description (programmatic access)
print("\n" + "=" * 60)
print("Method 5: Python .description (for programmatic use)")
print("=" * 60)

result = con.execute("SELECT * FROM employees LIMIT 0")
print("Column metadata:")
for col in result.description:
    print(f"  Name: {col[0]}, Type: {col[1]}")

In [ ]:
# Method 6: Pandas dtypes (when working with DataFrames)
print("\n" + "=" * 60)
print("Method 6: Pandas .dtypes (DataFrame schema)")
print("=" * 60)

df = con.execute("SELECT * FROM employees").df()
print(df.dtypes)

In [ ]:
# BONUS: Inspect nested Parquet schema (Staging layer example)
print("\n" + "=" * 60)
print("BONUS: Nested structure schema (Staging layer)")
print("=" * 60)

nested_schema = con.execute(f"""
    DESCRIBE SELECT * FROM read_parquet('{staging_path}')
""").df()

print(nested_schema)
print("\nNotice the STRUCT and LIST types - DuckDB preserves nested structures!")

### DuckDB vs Spark Schema Inspection

| Feature | Spark | DuckDB |
|---------|-------|---------|
| **Basic schema** | `df.printSchema()` | `DESCRIBE table` ✓ |
| **With statistics** | `df.describe()` | `SUMMARIZE table` ✓✓ Better! |
| **S3 without loading** | ❌ Need to load data | `DESCRIBE SELECT * FROM read_parquet('s3://...')` ✓ |
| **Nested types** | ✓ Shows | ✓ Shows (STRUCT, LIST, MAP) |
| **Programmatic** | `df.schema` | `.description` or `.df().dtypes` ✓ |

**Winner: DuckDB** 🏆
- `SUMMARIZE` gives schema + stats in one command
- Can inspect S3 files without downloading
- Cleaner SQL syntax

## Example 6: Raw vs Processed Data (Staging → DDS → DDM)

**Best Practice**: Don't flatten raw data! Save it nested, then process later.

### Data Lake Layers:
- **Staging**: Save API responses as-is (nested structures, raw data)
- **DDS** (Data Detail Store): Flatten, clean, deduplicate
- **DDM** (Data Mart Domain): Aggregated, business-ready tables

In [ ]:
# Step 1: STAGING LAYER - Save raw nested JSON to S3
# Keep the original structure! Parquet supports nested types.

# Simulate complex API response with nested data
raw_api_response = {
    "data": [
        {
            "order_id": 1001,
            "customer": {
                "id": 501,
                "name": "Alice Johnson",
                "email": "alice@example.com"
            },
            "items": [
                {"product": "Laptop", "quantity": 1, "price": 1200.00},
                {"product": "Mouse", "quantity": 2, "price": 25.00}
            ],
            "total": 1250.00,
            "order_date": "2024-10-01"
        },
        {
            "order_id": 1002,
            "customer": {
                "id": 502,
                "name": "Bob Smith",
                "email": "bob@example.com"
            },
            "items": [
                {"product": "Keyboard", "quantity": 1, "price": 85.00}
            ],
            "total": 85.00,
            "order_date": "2024-10-02"
        }
    ]
}

# Convert to DataFrame - pandas preserves nested structures!
raw_df = pd.DataFrame(raw_api_response['data'])

print("Raw nested data:")
print(raw_df.head())
print("\nData types (notice dict and list types):")
print(raw_df.dtypes)

In [ ]:
# Save nested data to Staging layer - NO FLATTENING!
staging_path = f"s3://{bucket_name}/staging/orders/raw_orders.parquet"

con.execute(f"""
    CREATE OR REPLACE TABLE staging_orders AS 
    SELECT 
        *,
        CURRENT_TIMESTAMP as ingested_at
    FROM raw_df
""")

con.execute(f"""
    COPY staging_orders 
    TO '{staging_path}' 
    (FORMAT PARQUET, COMPRESSION 'SNAPPY')
""")

print(f"✓ Raw nested data saved to Staging layer: {staging_path}")
print("  → Preserved nested customer and items structures!")

In [ ]:
# Step 2: Query nested Parquet directly from S3!
# DuckDB can access nested fields using dot notation

result = con.execute(f"""
    SELECT 
        order_id,
        customer.id as customer_id,
        customer.name as customer_name,
        customer.email as customer_email,
        total,
        order_date,
        items  -- Keep array as-is for now
    FROM read_parquet('{staging_path}')
""").df()

print("Querying nested Parquet from Staging layer:")
print(result)

In [ ]:
# Step 3: DDS LAYER - Flatten and process
# Now we transform nested → flat for analytics

dds_path = f"s3://{bucket_name}/dds/orders/orders_flattened.parquet"

con.execute(f"""
    CREATE OR REPLACE TABLE dds_orders AS
    SELECT 
        order_id,
        customer.id as customer_id,
        customer.name as customer_name,
        customer.email as customer_email,
        CAST(order_date AS DATE) as order_date,
        total as order_total,
        CAST(ingested_at AS TIMESTAMP) as ingested_at,
        CURRENT_TIMESTAMP as processed_at
    FROM read_parquet('{staging_path}')
""")

# Save to DDS layer
con.execute(f"""
    COPY dds_orders 
    TO '{dds_path}' 
    (FORMAT PARQUET, COMPRESSION 'SNAPPY')
""")

print(f"✓ Flattened data saved to DDS layer: {dds_path}")

# View processed data
con.execute("SELECT * FROM dds_orders").df()

In [ ]:
# Step 4: DDM LAYER - Create business-ready mart
# Aggregate and create analytical tables

ddm_path = f"s3://{bucket_name}/ddm/marts/customer_summary.parquet"

con.execute(f"""
    COPY (
        SELECT 
            customer_id,
            customer_name,
            customer_email,
            COUNT(*) as total_orders,
            SUM(order_total) as total_revenue,
            AVG(order_total) as avg_order_value,
            MIN(order_date) as first_order_date,
            MAX(order_date) as last_order_date
        FROM read_parquet('{dds_path}')
        GROUP BY customer_id, customer_name, customer_email
        ORDER BY total_revenue DESC
    )
    TO '{ddm_path}'
    (FORMAT PARQUET, COMPRESSION 'SNAPPY')
""")

print(f"✓ Mart saved to DDM layer: {ddm_path}")

# Query the mart
mart_result = con.execute(f"""
    SELECT * FROM read_parquet('{ddm_path}')
""").df()

print("\nCustomer Summary Mart:")
print(mart_result)

### Why This Layered Approach?

**Staging Layer Benefits:**
- ✅ **Preserve original data** - can always reprocess if logic changes
- ✅ **Audit trail** - know exactly what API returned
- ✅ **No data loss** - nested structures intact
- ✅ **Fast ingestion** - just save, no transformation
- ✅ **Parquet supports nested types** - no need to flatten

**DDS (Data Detail Store) Layer Benefits:**
- ✅ **Clean data** - deduplication, validation
- ✅ **Flattened** - easier to query
- ✅ **Type conversions** - proper dates, numbers
- ✅ **Standardized** - consistent schema
- ✅ **Business rules applied** - enriched and validated

**DDM (Data Mart Domain) Layer Benefits:**
- ✅ **Business-ready** - aggregated metrics
- ✅ **Fast queries** - pre-computed
- ✅ **Denormalized** - optimized for analytics
- ✅ **Self-service ready** - BI tools can consume directly

**For your use case:**
```
API JSON → Staging (nested Parquet) → DDS (flat Parquet) → DDM (marts)
```

This is a proven data lakehouse pattern! 🎯

In [ ]:
# Close DuckDB connection
con.close()
print("✓ DuckDB connection closed")

## Summary

This notebook demonstrated:

1. ✓ **S3 Configuration**: Set up DuckDB with Wasabi credentials
2. ✓ **Write to S3**: Save DataFrames as Parquet on S3
3. ✓ **API to S3**: Fetch JSON from APIs and save to S3
4. ✓ **Query S3**: Read and analyze Parquet files directly from S3
5. ✓ **Create Marts**: Build analytical datasets from S3 data
6. ✓ **Layered Architecture**: Staging (raw) → DDS (processed) → DDM (marts)
7. ✓ **Schema Inspection**: Multiple ways to view schemas

### Key Advantages of DuckDB:
- ⚡ **Fast** - 10-100x faster than Spark for small-to-medium data
- 🎯 **Direct S3 queries** - no need to load data into memory
- 🪶 **Lightweight** - runs in-process, no cluster needed
- 🐍 **Pure Python** - easy to integrate with Airflow
- 💰 **Cost-effective** - minimal infrastructure
- 📦 **Nested data support** - query Parquet with nested structures

### Best Practices for Data Lakehouse:
1. **Staging layer** - Save raw data as-is, preserve original structure
2. **DDS layer** - Process and flatten, clean and standardize
3. **DDM layer** - Create marts, aggregate for analytics
4. **Use Parquet** - Columnar, compressed, supports nested types
5. **Query in-place** - DuckDB reads S3 directly, no copying needed

### Architecture Pattern:
```
API (JSON) → Staging (nested Parquet) → DDS (flat Parquet) → DDM (aggregated marts)
```